# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [1]:
#Import your libraries

import numpy as np
import pandas as pd
import re

# Introduction

In this lab, we will use two datasets. Both datasets contain variables that describe apps from the Google Play Store. We will use our knowledge in feature extraction to process these datasets and prepare them for the use of a ML algorithm.

# Challenge 1 - Loading and Extracting Features from the First Dataset

#### In this challenge, our goals are: 

* Exploring the dataset.
* Identify the columns with missing values.
* Either replacing the missing values in each column or drop the columns.
* Conver each column to the appropriate type.

#### The first dataset contains different information describing the apps. 

Load the dataset into the variable `google_play` in the cell below. The dataset is in the file `googleplaystore.csv`

In [2]:
# Your code here:

google_play = pd.read_csv('../googleplaystore.csv')

#### Examine all variables and their types in the following cell

In [3]:
# Your code here:
google_play.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

#### Since this dataset only contains one numeric column, let's skip the `describe()` function and look at the first 5 rows using the `head()` function

In [4]:
# Your code here:
google_play.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


#### We can see that there are a few columns that could be coerced to numeric.

Start with the reviews column. We can evaluate what value is causing this column to be of object type finding the non-numeric values in this column. To do this, we recall the `to_numeric()` function. With this function, we are able to coerce all non-numeric data to null. We can then use the `isnull()` function to subset our dataframe using the True/False column that this function generates.

In the cell below, transform the Reviews column to numeric and assign this new column to the variable `Reviews_numeric`. Make sure to coerce the errors.

In [6]:
# Your code here:
def numeric(strg):
    num = pd.to_numeric(strg, errors='coerce')
    return num

google_play['Reviews_numeric'] = google_play['Reviews'].apply(numeric)

Next, create a column containing True/False values using the `isnull()` function. Assign this column to the `Reviews_isnull` variable.

In [7]:
# Your code here:
google_play['Reviews_isnull'] = google_play['Reviews_numeric'].isnull()

Finally, subset the `google_play` with `Reviews_isnull`. This should give you all the rows that contain non-numeric characters.

Your output should look like:

![Reviews_bool.png](reviews-bool.png)

In [8]:
# Your code here:
google_play.loc[google_play['Reviews_isnull']==True]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Reviews_numeric,Reviews_isnull
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN,NaN,True


#### We see that Google Play is using a shorthand for millions. 

Let's write a function to transform this data.

Steps:

1. Create a function that returns the correct numeric values of *Reviews*.
1. Define a test string with `M` in the last character.
1. Test your function with the test string. Make sure your function works correctly. If not, modify your functions and test again.

In [9]:
# Your code here

def convert_string_to_numeric(s):
    """
    Convert a string value to numeric. If the last character of the string is `M`, obtain the 
    numeric part of the string, multiply it with 1,000,000, then return the result. Otherwise, 
    convert the string to numeric value and return the result.
    
    Args:
        s: The Reviews score in string format.

    Returns:
        The correct numeric value of the Reviews score.
    """
    if s.endswith('M'):
        string = re.findall(r"[-+]?\d*\.\d+|\d+", s)[0]
        number = float(string)
        return number*1000000
    else:
        floated = float(s)
        return floated


test_string = '4.0M'

convert_string_to_numeric(test_string) == 4000000

True

The last step is to apply the function to the `Reviews` column in the following cell:

In [10]:
# Your code here:
google_play['Reviews'] = google_play['Reviews'].apply(convert_string_to_numeric)

Check the non-numeric `Reviews` row again. It should have been fixed now and you should see:

![Reviews_bool_fixed.png](reviews-bool-fixed.png)

In [11]:
# Your code here
google_play.loc[google_play['Reviews_isnull']==True]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Reviews_numeric,Reviews_isnull
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3000000.0,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN,NaN,True


Also check the variable types of `google_play`. The `Reviews` column should be a `float64` type now.

In [12]:
# Your code here:
google_play.dtypes

App                 object
Category            object
Rating             float64
Reviews            float64
Size                object
Installs            object
Type                object
Price               object
Content Rating      object
Genres              object
Last Updated        object
Current Ver         object
Android Ver         object
Reviews_numeric    float64
Reviews_isnull        bool
dtype: object

#### The next column we will look at is `Size`. We start by looking at all unique values in `Size`:

*Hint: use `unique()` ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html))*.

In [13]:
# Your code here:
google_play['Size'].unique()

array(['19M', '14M', '8.7M', '25M', '2.8M', '5.6M', '29M', '33M', '3.1M',
       '28M', '12M', '20M', '21M', '37M', '2.7M', '5.5M', '17M', '39M',
       '31M', '4.2M', '7.0M', '23M', '6.0M', '6.1M', '4.6M', '9.2M',
       '5.2M', '11M', '24M', 'Varies with device', '9.4M', '15M', '10M',
       '1.2M', '26M', '8.0M', '7.9M', '56M', '57M', '35M', '54M', '201k',
       '3.6M', '5.7M', '8.6M', '2.4M', '27M', '2.5M', '16M', '3.4M',
       '8.9M', '3.9M', '2.9M', '38M', '32M', '5.4M', '18M', '1.1M',
       '2.2M', '4.5M', '9.8M', '52M', '9.0M', '6.7M', '30M', '2.6M',
       '7.1M', '3.7M', '22M', '7.4M', '6.4M', '3.2M', '8.2M', '9.9M',
       '4.9M', '9.5M', '5.0M', '5.9M', '13M', '73M', '6.8M', '3.5M',
       '4.0M', '2.3M', '7.2M', '2.1M', '42M', '7.3M', '9.1M', '55M',
       '23k', '6.5M', '1.5M', '7.5M', '51M', '41M', '48M', '8.5M', '46M',
       '8.3M', '4.3M', '4.7M', '3.3M', '40M', '7.8M', '8.8M', '6.6M',
       '5.1M', '61M', '66M', '79k', '8.4M', '118k', '44M', '695k', '1.6M',
     

You should have seen lots of unique values of the app sizes.

#### While we can convert most of the `Size` values to numeric in the same way we converted the `Reviews` values, there is one value that is impossible to convert.

What is that badass value? Enter it in the next cell and calculate the proportion of its occurence to the total number of records of `google_play`.

In [14]:
# Your code here:
'The badass value is {} and the proportion of rows containing it is {}%'.format('1,000+',\
        ((google_play[google_play.Size == '1,000+'].shape[0])/(google_play.shape[0])*100))

'The badass value is 1,000+ and the proportion of rows containing it is 0.009224241306152568%'

#### While this column may be useful for other types of analysis, we opt to drop it from our dataset. 

There are two reasons. First, the majority of the data are ordinal but a sizeable proportion are missing because we cannot convert them to numerical values. Ordinal data are both numerical and categorical, and they usually can be ranked (e.g. 82k is smaller than 91M). In contrast, non-ordinal categorical data such as blood type and eye color cannot be ranked. The second reason is as a categorical column, it has too many unique values to produce meaningful insights. Therefore, in our case the simplest strategy would be to drop the column.

Drop the column in the cell below (use `inplace=True`)

In [15]:
# Your code here:
google_play.drop(['Size'],axis=1,inplace=True)

#### Now let's look at how many missing values are in each column. 

This will give us an idea of whether we should come up with a missing data strategy or give up on the column all together. In the next column, find the number of missing values in each column: 

*Hint: use the `isna()` and `sum()` functions.*

In [16]:
# Your code here:
google_play.isna().sum()

App                   0
Category              0
Rating             1474
Reviews               0
Installs              0
Type                  1
Price                 0
Content Rating        1
Genres                0
Last Updated          0
Current Ver           8
Android Ver           3
Reviews_numeric       1
Reviews_isnull        0
dtype: int64

You should find the column with the most missing values is now `Rating`.

#### What is the proportion of the missing values in `Rating` to the total number of records?

Enter your answer in the cell below.

In [17]:
# Your code here:
'The proportion of rows with missing values in Rating is {}%'\
.format((google_play['Rating'].isna().sum())/(google_play.shape[0])*100)

'The proportion of rows with missing values in Rating is 13.596531685268886%'

A sizeable proportion of the `Rating` column is missing. A few other columns also contain several missing values.

#### We opt to preserve these columns and remove the rows containing missing data.

In particular, we don't want to drop the `Rating` column because:

* It is one of the most important columns in our dataset. 

* Since the dataset is not a time series, the loss of these rows will not have a negative impact on our ability to analyze the data. It will, however, cause us to lose some meaningful observations. But the loss is limited compared to the gain we receive by preserving these columns.

In the cell below, remove all rows containing at least one missing value. Use the `dropna()` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html)). Assign the new dataframe to the variable `google_missing_removed`.

In [18]:
# Your code here:
google_missing_removed = google_play.dropna()

From now on, we use the `google_missing_removed` variable instead of `google_play`.

#### Next, we look at the `Last Updated` column.

The `Last Updated` column seems to contain a date, though it is classified as an object type. Let's convert this column using the `pd.to_datetime` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)).

In [19]:
# Your code here:
google_missing_removed['Last Updated'] = pd.to_datetime(google_missing_removed['Last Updated'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### The last column we will transform is `Price`. 

We start by looking at the unique values of this column.

In [20]:
# Your code here:
google_play['Price'].unique()

array(['0', '$4.99', '$3.99', '$6.99', '$1.49', '$2.99', '$7.99', '$5.99',
       '$3.49', '$1.99', '$9.99', '$7.49', '$0.99', '$9.00', '$5.49',
       '$10.00', '$24.99', '$11.99', '$79.99', '$16.99', '$14.99',
       '$1.00', '$29.99', '$12.99', '$2.49', '$10.99', '$1.50', '$19.99',
       '$15.99', '$33.99', '$74.99', '$39.99', '$3.95', '$4.49', '$1.70',
       '$8.99', '$2.00', '$3.88', '$25.99', '$399.99', '$17.99',
       '$400.00', '$3.02', '$1.76', '$4.84', '$4.77', '$1.61', '$2.50',
       '$1.59', '$6.49', '$1.29', '$5.00', '$13.99', '$299.99', '$379.99',
       '$37.99', '$18.99', '$389.99', '$19.90', '$8.49', '$1.75',
       '$14.00', '$4.85', '$46.99', '$109.99', '$154.99', '$3.08',
       '$2.59', '$4.80', '$1.96', '$19.40', '$3.90', '$4.59', '$15.46',
       '$3.04', '$4.29', '$2.60', '$3.28', '$4.60', '$28.99', '$2.95',
       '$2.90', '$1.97', '$200.00', '$89.99', '$2.56', '$30.99', '$3.61',
       '$394.99', '$1.26', 'Everyone', '$1.20', '$1.04'], dtype=object)

Since all prices are ordinal data without exceptions, we can tranform this column by removing the dollar sign and converting to numeric. We can create a new column called `Price Numerical` and drop the original column.

We will achieve our goal in three steps. Follow the instructions of each step below.

#### First we remove the dollar sign. Do this in the next cell by applying the `str.replace` function to the column to replace `$` with an empty string (`''`).

In [21]:
# Your code here:
google_missing_removed['Price Numerical'] = google_missing_removed['Price'].str.replace('$','')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### Second step, coerce the `Price Numerical` column to numeric.

In [22]:
# Your code here:
google_missing_removed['Price Numerical'] = google_missing_removed['Price Numerical'].apply(numeric)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


**Finally, drop the original `Price` column.**

In [23]:
# Your code here:
google_missing_removed.drop(['Price'],axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Now check the variable types of `google_missing_removed`. Make sure:

* `Size` and `Price` columns have been removed.
* `Rating`, `Reviews`, and `Price Numerical` have the type of `float64`.
* `Last Updated` has the type of `datetime64`.

In [24]:
# Your code here
google_missing_removed.dtypes

App                        object
Category                   object
Rating                    float64
Reviews                   float64
Installs                   object
Type                       object
Content Rating             object
Genres                     object
Last Updated       datetime64[ns]
Current Ver                object
Android Ver                object
Reviews_numeric           float64
Reviews_isnull               bool
Price Numerical           float64
dtype: object

# Challenge 2 - Loading and Extracting Features from the Second Dataset

Load the second dataset to the variable `google_reviews`. The data is in the file `googleplaystore_user_reviews.csv`.

In [25]:
# Your code here:

google_review = pd.read_csv('../googleplaystore_user_reviews.csv')

#### This dataset contains the top 100 reviews for each app. 

Let's examine this dataset using the `head` function

In [26]:
# Your code here
google_review.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


#### The main piece of information we would like to extract from this dataset is the proportion of positive reviews of each app. 

Columns like `Sentiment_Polarity` and `Sentiment_Subjectivity` are not to our interests because we have no clue how to use them. We do not care about `Translated_Review` because natural language processing is too complex for us at present (in fact the `Sentiment`, `Sentiment_Polarity`, and `Sentiment_Subjectivity` columns are derived from `Translated_Review` the data scientists). 

What we care about in this challenge is `Sentiment`. To be more precise, we care about **what is the proportion of *Positive* sentiment of each app**. This will require us to aggregate the `Sentiment` data by `App` in order to calculate the proportions.

Now that you are clear about what we are trying to achieve, follow the steps below that will walk you through towards our goal.

#### Our first step will be to remove all rows with missing sentiment. 

In the next cell, drop all rows with missing data using the `dropna()` function and assign this new dataframe to `review_missing_removed`.

In [27]:
# Your code here:
review_missing_removed = google_review.dropna()

#### Now, use the `value_counts()` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html)) to get a sense on how many apps are in this dataset and their review counts.

In [28]:
# Your code here:
print('The total number of App is {}'.format(review_missing_removed['App'].value_counts().shape[0]))
display(review_missing_removed['App'].value_counts())

The total number of App is 865


Bowmasters                                            312
Angry Birds Classic                                   273
Helix Jump                                            273
Calorie Counter - MyFitnessPal                        254
Duolingo: Learn Languages Free                        240
Candy Crush Saga                                      240
Garena Free Fire                                      222
8 Ball Pool                                           219
Calorie Counter - Macros                              200
10 Best Foods for You                                 194
CBS Sports App - Scores, News, Stats & Watch Live     192
Google Photos                                         191
Alto's Adventure                                      175
8fit Workouts & Meal Planner                          171
DRAGON BALL LEGENDS                                   167
Candy Crush Soda Saga                                 166
Clash Royale                                          165
Adobe Acrobat 

#### Now the tough part comes. Let's plan how we will achieve our goal:

1. We will count the number of reviews that contain *Positive* in the `Sentiment` column.

1. We will create a new dataframe to contain the `App` name, the number of positive reviews, and the total number of reviews of each app.

1. We will then loop the new dataframe to calculate the postivie review portion of each app.

#### Step 1: Count the number of positive reviews.

In the following cell, write a function that takes a column and returns the number of times *Positive* appears in the column. 

*Hint: One option is to use the `np.where()` function ([documentation](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.where.html)).*

In [29]:
# Your code below

def positive_function(x):
    """
    Count how many times the string `Positive` appears in a column (exact string match).
    
    Args:
        x: data column
    
    Returns:
        The number of occurrences of `Positive` in the column data.
    """
    
    return len(np.where(x=='Positive')[0])

#### Step 2: Create a new dataframe to contain the `App` name, the number of positive reviews, and the total number of reviews of each app

We will group `review_missing_removed` by the `App` column, then aggregate the grouped dataframe on the number of positive reviews and the total review counts of each app. The result will be assigned to a new variable `google_agg`. Here is the ([documentation on how to achieve it](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.agg.html)). Take a moment or two to read the documentation and google examples because it is pretty complex.

When you obtain `google_agg`, check its values to make sure it has an `App` column as its index as well as a `Positive` column and a `Total` column. Your output should look like:

![Positive Reviews Agg](positive-review-agg.png)

*Hint: Use `positive_function` you created earlier as part of the param passed to the `agg()` function in order to aggregate the number of positive reviews.*

#### Bonus:

As of Pandas v0.23.4, you may opt to supply an array or an object to `agg()`. If you use the array param, you'll need to rename the columns so that their names are `Positive` and `Total`. Using the object param will allow you to create the aggregated columns with the desirable names without renaming them. However, you will probably encounter a warning indicating supplying an object to `agg()` will become outdated. It's up to you which way you will use. Try both ways out. Any way is fine as long as it works.

In [30]:
# Your code here:
google_agg = review_missing_removed.groupby('App').\
agg({'Sentiment':lambda x: positive_function(x),'App':lambda x: x.value_counts()})
google_agg.rename(columns={'Sentiment':'Positive','App':'Total'},inplace=True)

Print the first 5 rows of `google_agg` to check it.

In [31]:
# Your code here
google_agg.head()

,Positive,Total
App,,
10 Best Foods for You,162,194
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,31,40
11st,23,39
1800 Contacts - Lens Store,64,80
1LINE – One Line with One Touch,27,38


#### Add a derived column to `google_agg` that is the ratio of the `Positive` and the `Total` columns. Call this column `Positive Ratio`. 

Make sure to account for the case where the denominator is zero using the `np.where()` function.

In [32]:
# Your code here:
google_agg['Positive Ratio'] = google_agg['Positive']/google_agg['Total']
display(review_missing_removed['Sentiment'].unique())
display(review_missing_removed['Sentiment'].isna().sum())
display(review_missing_removed['Sentiment'].isnull().sum())

array(['Positive', 'Neutral', 'Negative'], dtype=object)

0

0

**Nunca se va a dividir por cero ya que la columna Total no tiene valor cero ya que es la cuenta de cuantas 
veces cada una de las App está listada en el dataframe que como mínimo es 1. Además, se puede verificar que 
no existen valores vacios en la columna de sentimiento de manera que no puede haber división entre cero.**

#### Now drop the `Positive` and `Total` columns. Do this with `inplace=True`.

In [33]:
# Your code here:
google_agg.drop(['Positive','Total'],axis=1,inplace=True)

Print the first 5 rows of `google_agg`. Your output should look like:

![Positive Reviews Agg](positive-review-ratio.png)

In [34]:
# Your code here:
google_agg.head()

,Positive Ratio
App,
10 Best Foods for You,0.835052
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.775000
11st,0.589744
1800 Contacts - Lens Store,0.800000
1LINE – One Line with One Touch,0.710526


# Challenge 3 - Join the Dataframes

In this part of the lab, we will join the two dataframes and obtain a dataframe that contains features we can use in our ML algorithm.

In the next cell, join the `google_missing_removed` dataframe with the `google_agg` dataframe on the `App` column. Assign this dataframe to the variable `google`.

In [35]:
google_agg.shape

(865, 1)

In [36]:
google_missing_removed.shape

(9360, 14)

In [37]:
# Your code here:
google = pd.merge(google_missing_removed,google_agg,on='App')

#### Let's look at the final result using the `head()` function. Your final product should look like:

![Final Product](google-final-head.png)

In [38]:
# Your code here:
google.shape

(1228, 15)

In [39]:
google.head()

,App,Category,Rating,Reviews,Installs,Type,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Reviews_numeric,Reviews_isnull,Price Numerical,Positive Ratio
0,Coloring book moana,ART_AND_DESIGN,3.9,967.0,"500,000+",Free,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,967.0,False,0.0,0.590909
1,Coloring book moana,FAMILY,3.9,974.0,"500,000+",Free,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,974.0,False,0.0,0.590909
2,Garden Coloring Book,ART_AND_DESIGN,4.4,13791.0,"1,000,000+",Free,Everyone,Art & Design,2017-09-20,2.9.2,3.0 and up,13791.0,False,0.0,0.711111
3,FlipaClip - Cartoon animation,ART_AND_DESIGN,4.3,194216.0,"5,000,000+",Free,Everyone,Art & Design,2018-08-03,2.2.5,4.0.3 and up,194216.0,False,0.0,1.000000
4,Boys Photo Editor - Six Pack & Men's Suit,ART_AND_DESIGN,4.1,654.0,"100,000+",Free,Everyone,Art & Design,2018-03-20,1.1,4.0.3 and up,654.0,False,0.0,0.605263


## Bonus

In [84]:
google_bonus = google.drop(['Last Updated','Current Ver','Android Ver'],axis=1)

In [85]:
google_bonus.head()

,App,Category,Rating,Reviews,Installs,Type,Content Rating,Genres,Reviews_numeric,Reviews_isnull,Price Numerical,Positive Ratio
0,Coloring book moana,ART_AND_DESIGN,3.9,967.0,"500,000+",Free,Everyone,Art & Design;Pretend Play,967.0,False,0.0,0.590909
1,Coloring book moana,FAMILY,3.9,974.0,"500,000+",Free,Everyone,Art & Design;Pretend Play,974.0,False,0.0,0.590909
2,Garden Coloring Book,ART_AND_DESIGN,4.4,13791.0,"1,000,000+",Free,Everyone,Art & Design,13791.0,False,0.0,0.711111
3,FlipaClip - Cartoon animation,ART_AND_DESIGN,4.3,194216.0,"5,000,000+",Free,Everyone,Art & Design,194216.0,False,0.0,1.000000
4,Boys Photo Editor - Six Pack & Men's Suit,ART_AND_DESIGN,4.1,654.0,"100,000+",Free,Everyone,Art & Design,654.0,False,0.0,0.605263


In [86]:
google_bonus.dtypes

App                 object
Category            object
Rating             float64
Reviews            float64
Installs            object
Type                object
Content Rating      object
Genres              object
Reviews_numeric    float64
Reviews_isnull        bool
Price Numerical    float64
Positive Ratio     float64
dtype: object

In [87]:
google_bonus['Category'].unique()

array(['ART_AND_DESIGN', 'FAMILY', 'AUTO_AND_VEHICLES', 'BEAUTY',
       'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMMUNICATION', 'COMICS',
       'DATING', 'EDUCATION', 'ENTERTAINMENT', 'EVENTS', 'FINANCE',
       'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME',
       'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'GAME', 'SPORTS', 'MEDICAL',
       'SOCIAL', 'SHOPPING', 'PHOTOGRAPHY', 'TRAVEL_AND_LOCAL', 'TOOLS',
       'PERSONALIZATION', 'PRODUCTIVITY', 'PARENTING', 'WEATHER',
       'VIDEO_PLAYERS', 'NEWS_AND_MAGAZINES', 'MAPS_AND_NAVIGATION'],
      dtype=object)

In [88]:
google_bonus['Installs'].unique()

array(['500,000+', '1,000,000+', '5,000,000+', '100,000+', '10,000,000+',
       '100,000,000+', '50,000,000+', '10,000+', '1,000,000,000+',
       '500,000,000+', '50,000+', '1,000+'], dtype=object)

In [89]:
google_bonus['Type'].unique()

array(['Free', 'Paid'], dtype=object)

In [90]:
google_bonus['Content Rating'].unique()

array(['Everyone', 'Teen', 'Mature 17+', 'Everyone 10+',
       'Adults only 18+'], dtype=object)

In [91]:
google_bonus['Genres'].unique()

array(['Art & Design;Pretend Play', 'Art & Design',
       'Art & Design;Creativity', 'Auto & Vehicles', 'Beauty',
       'Books & Reference', 'Business', 'Communication', 'Comics',
       'Dating', 'Education;Education', 'Education', 'Entertainment',
       'Events', 'Finance', 'Food & Drink', 'Health & Fitness',
       'House & Home', 'Libraries & Demo', 'Lifestyle', 'Casual',
       'Puzzle;Brain Games', 'Arcade', 'Casual;Pretend Play', 'Strategy',
       'Action', 'Puzzle', 'Sports', 'Casual;Creativity', 'Racing',
       'Board', 'Simulation', 'Adventure', 'Role Playing',
       'Simulation;Education', 'Action;Action & Adventure',
       'Casual;Brain Games', 'Simulation;Action & Adventure',
       'Educational;Creativity', 'Educational;Education',
       'Educational;Pretend Play', 'Racing;Action & Adventure',
       'Entertainment;Music & Video', 'Educational;Brain Games',
       'Casual;Action & Adventure', 'Education;Pretend Play',
       'Simulation;Pretend Play', 'Sports;Acti

In [92]:
google_bonus.drop(['Content Rating','Genres','Reviews_isnull'],axis=1,inplace=True)
google_bonus.head()

,App,Category,Rating,Reviews,Installs,Type,Reviews_numeric,Price Numerical,Positive Ratio
0,Coloring book moana,ART_AND_DESIGN,3.9,967.0,"500,000+",Free,967.0,0.0,0.590909
1,Coloring book moana,FAMILY,3.9,974.0,"500,000+",Free,974.0,0.0,0.590909
2,Garden Coloring Book,ART_AND_DESIGN,4.4,13791.0,"1,000,000+",Free,13791.0,0.0,0.711111
3,FlipaClip - Cartoon animation,ART_AND_DESIGN,4.3,194216.0,"5,000,000+",Free,194216.0,0.0,1.000000
4,Boys Photo Editor - Six Pack & Men's Suit,ART_AND_DESIGN,4.1,654.0,"100,000+",Free,654.0,0.0,0.605263


In [93]:
google_bonus['Free_Paid'] = pd.get_dummies(google_bonus.Type,drop_first=True)
google_bonus['Free_Paid'].sum()

15

In [94]:
def str_to_num(install):
    string = install.replace(',','').replace('+','')
    number = float(string)
    return number

str_to_num('500,000+') == 500000

True

In [95]:
google_bonus['Installs'] = google_bonus['Installs'].apply(str_to_num)
google_bonus.shape

(1228, 10)

In [96]:
category = pd.get_dummies(google_bonus.Category, drop_first=True)
google_cat = pd.concat([google_bonus, category], axis=1)
google_cat.head()

,App,Category,Rating,Reviews,Installs,Type,Reviews_numeric,Price Numerical,Positive Ratio,Free_Paid,...,PERSONALIZATION,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,Coloring book moana,ART_AND_DESIGN,3.9,967.0,500000.0,Free,967.0,0.0,0.590909,0,...,0,0,0,0,0,0,0,0,0,0
1,Coloring book moana,FAMILY,3.9,974.0,500000.0,Free,974.0,0.0,0.590909,0,...,0,0,0,0,0,0,0,0,0,0
2,Garden Coloring Book,ART_AND_DESIGN,4.4,13791.0,1000000.0,Free,13791.0,0.0,0.711111,0,...,0,0,0,0,0,0,0,0,0,0
3,FlipaClip - Cartoon animation,ART_AND_DESIGN,4.3,194216.0,5000000.0,Free,194216.0,0.0,1.000000,0,...,0,0,0,0,0,0,0,0,0,0
4,Boys Photo Editor - Six Pack & Men's Suit,ART_AND_DESIGN,4.1,654.0,100000.0,Free,654.0,0.0,0.605263,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
google_ml = google_cat.drop(['Category','Type'],axis=1)._get_numeric_data()
google_ml.rename(columns={'Positive Ratio':'PR'},inplace=True)

In [98]:
def float_to_int(floated):
    integer = int(floated)
    return integer

float_to_int(500000.0) == 500000

True

In [104]:
google_ml['Rating'] = google_ml['Rating'].apply(float_to_int)
google_ml['Reviews'] = google_ml['Reviews'].apply(float_to_int)
google_ml['Installs'] = google_ml['Installs'].apply(float_to_int)
google_ml['Reviews_numeric'] = google_ml['Reviews_numeric'].apply(float_to_int)

In [105]:
display(google_ml.shape,google_ml.columns,google_ml.dtypes,google_ml.head())

(1228, 39)

Index(['Rating', 'Reviews', 'Installs', 'Reviews_numeric', 'Price Numerical',
       'PR', 'Free_Paid', 'AUTO_AND_VEHICLES', 'BEAUTY', 'BOOKS_AND_REFERENCE',
       'BUSINESS', 'COMICS', 'COMMUNICATION', 'DATING', 'EDUCATION',
       'ENTERTAINMENT', 'EVENTS', 'FAMILY', 'FINANCE', 'FOOD_AND_DRINK',
       'GAME', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME', 'LIBRARIES_AND_DEMO',
       'LIFESTYLE', 'MAPS_AND_NAVIGATION', 'MEDICAL', 'NEWS_AND_MAGAZINES',
       'PARENTING', 'PERSONALIZATION', 'PHOTOGRAPHY', 'PRODUCTIVITY',
       'SHOPPING', 'SOCIAL', 'SPORTS', 'TOOLS', 'TRAVEL_AND_LOCAL',
       'VIDEO_PLAYERS', 'WEATHER'],
      dtype='object')

Rating                   int64
Reviews                  int64
Installs                 int64
Reviews_numeric          int64
Price Numerical        float64
PR                     float64
Free_Paid                uint8
AUTO_AND_VEHICLES        uint8
BEAUTY                   uint8
BOOKS_AND_REFERENCE      uint8
BUSINESS                 uint8
COMICS                   uint8
COMMUNICATION            uint8
DATING                   uint8
EDUCATION                uint8
ENTERTAINMENT            uint8
EVENTS                   uint8
FAMILY                   uint8
FINANCE                  uint8
FOOD_AND_DRINK           uint8
GAME                     uint8
HEALTH_AND_FITNESS       uint8
HOUSE_AND_HOME           uint8
LIBRARIES_AND_DEMO       uint8
LIFESTYLE                uint8
MAPS_AND_NAVIGATION      uint8
MEDICAL                  uint8
NEWS_AND_MAGAZINES       uint8
PARENTING                uint8
PERSONALIZATION          uint8
PHOTOGRAPHY              uint8
PRODUCTIVITY             uint8
SHOPPING

,Rating,Reviews,Installs,Reviews_numeric,Price Numerical,PR,Free_Paid,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,...,PERSONALIZATION,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,3,967,500000,967,0.0,0.590909,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,974,500000,974,0.0,0.590909,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,13791,1000000,13791,0.0,0.711111,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,194216,5000000,194216,0.0,1.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,654,100000,654,0.0,0.605263,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
from sklearn.model_selection import train_test_split

In [107]:
y = google_ml.Rating
X = google_ml.drop(['Rating'], axis=1)

In [108]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [109]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(982, 38) (246, 38) (982,) (246,)


In [110]:
from sklearn import svm
rgs = svm.SVR()
display(rgs.fit(X_train, y_train))
display(rgs.score(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

0.3447497173858983

In [112]:
#Verificación
rgs.predict(X_test) - y_test

# Tabla Error Difference

df1 = pd.DataFrame(np.array([rgs.predict(X_test),y_test,rgs.predict(X_test) - y_test]).T)
df1.columns = ['Predicted Rating',"GrounTruth","Error Difference"]

df2 = pd.DataFrame(np.abs(rgs.predict(X_test) - y_test)) # en números absolutos

display(df1.head(), df2.describe())

,Predicted Rating,GrounTruth,Error Difference
0,3.900083,4.0,-0.099917
1,3.820471,4.0,-0.179529
2,3.899927,4.0,-0.100073
3,3.899923,4.0,-0.100077
4,3.076188,3.0,0.076188


,Rating
count,246.000000
mean,0.213640
std,0.199523
min,0.015965
25%,0.100148
50%,0.179529
75%,0.179529
max,0.828082


In [113]:
#Entrenar más modelos (repetir y luego hacer media de las medias: crosscorrelation):


In [ ]:
from sklearn import preprocessing
from sklearn import utils
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y_train)
print(utils.multiclass.type_of_target(y_train))
print(utils.multiclass.type_of_target(y_train.astype('int')))
print(utils.multiclass.type_of_target(encoded))